
# Proyecto análisis del rendimiento de las escuelas públicas de Chicago y datos de crimen en 2011-2012


## Descripción

#### El objetivo fue analizar tres bases de datos de Chicago (censo 2011, datos de crimen, datos escolares 2011-2012). Usando consultas SQL, se identificó las 10 escuelas con mayor asistencia, se evaluó la seguridad de las escuelas basada en la tasa de crimen, y se analizó la asistencia escolar baja (<70%). Esto ayudó a identificar áreas comunitarias críticas que necesitan atención estatal prioritaria.






## Chicago Public Schools - Progress Report Cards (2011-2012)

La ciudad de Chicago publicó un dataset que muestra todos los datos de rendimiento a nivel escolar utilizados para crear Progress report cards escolares para el año escolar 2011-2012. El conjunto de datos está disponible en el Portal de datos de Chicago: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

Link de database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&download=true&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTA**:

Esta base de datos de la ciudad de Chicago es la que usaremos. El link es largo así que dar clic aquí: <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01">link</a>.




### Conectarse a la base de datos



##### Se usará esta sintaxis para usar magic sql en sql lite

  **%sql sqlite://DatabaseName**   
  
donde DatabaseName será el archivo **.db** 


In [2]:
import csv, sqlite3

con = sqlite3.connect("RealWorldData.db")
cur = con.cursor()

In [3]:

!pip install -q pandas==1.1.5

In [4]:
%load_ext sql

In [6]:
%sql sqlite:///RealWorldData.db

'Connected: @RealWorldData.db'

### Almacenar el dataset en una tabla




In [7]:

import pandas
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [8]:
# Escribir la consulta para recuperar la lista de todas las tablas en la database
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///RealWorldData.db
Done.


name
CENSUS_DATA
CHICAGO_CRIME_DATA
CHICAGO_PUBLIC_SCHOOLS_DATA


In [1]:
# Se escribe la consulta para recuperar el número de columnas en la tabla SCHOOLS
%sql SELECT count(name) FROM PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS_DATA');

UsageError: Line magic function `%sql` not found.


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.


In [2]:
# Se escribe la consulta para recuperar todos los nombres de las columnas en la tabla SCHOOLS junto con sus tipos de datos y longitud
%sql SELECT name,type,length(type) FROM PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS_DATA');

UsageError: Line magic function `%sql` not found.




### Número de escuelas en el dataset



In [13]:
%sql SELECT COUNT(*) FROM CHICAGO_PUBLIC_SCHOOLS_DATA WHERE "Elementary, Middle, or High School" = 'ES';

 * sqlite:///RealWorldData.db
Done.


COUNT(*)
462


### El safe score (puntuación de seguridad) más alto




In [14]:
%sql SELECT MAX(SAFETY_SCORE) FROM CHICAGO_PUBLIC_SCHOOLS_DATA;


 * sqlite:///RealWorldData.db
Done.


MAX(SAFETY_SCORE)
99.0


### Escuelas con el safe score más alto



In [18]:
%sql SELECT NAME_OF_SCHOOL,SAFETY_SCORE FROM CHICAGO_PUBLIC_SCHOOLS_DATA WHERE SAFETY_SCORE = (SELECT MAX(SAFETY_SCORE) FROM CHICAGO_PUBLIC_SCHOOLS_DATA)

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,SAFETY_SCORE
Abraham Lincoln Elementary School,99.0
Alexander Graham Bell Elementary School,99.0
Annie Keller Elementary Gifted Magnet School,99.0
Augustus H Burley Elementary School,99.0
Edgar Allan Poe Elementary Classical School,99.0
Edgebrook Elementary School,99.0
Ellen Mitchell Elementary School,99.0
James E McDade Elementary Classical School,99.0
James G Blaine Elementary School,99.0
LaSalle Elementary Language Academy,99.0


### 10 mejores colegios con la "asistencia estudiantil promedio" más alta


In [19]:
%sql SELECT NAME_OF_SCHOOL, Average_Teacher_Attendance FROM CHICAGO_PUBLIC_SCHOOLS_DATA ORDER BY Average_Teacher_Attendance DESC LIMIT 10

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,Average_Teacher_Attendance
Lazaro Cardenas Elementary School,98.50%
George W Curtis Elementary School,98.10%
Louis Armstrong Math & Science Elementary School,97.90%
Lenart Elementary Regional Gifted Center,97.80%
Mark Twain Elementary School,97.80%
Skinner North,97.80%
Edward Everett Elementary School,97.70%
Ellen Mitchell Elementary School,97.70%
James Madison Elementary School,97.70%
Joseph Brennemann Elementary School,97.70%


### La lista de 5 escuelas con el promedio de asistencia estudiantil más bajo ordenadas en orden ascendente según la asistencia




In [20]:
%sql SELECT NAME_OF_SCHOOL,Average_Teacher_Attendance FROM CHICAGO_PUBLIC_SCHOOLS_DATA ORDER BY Average_Teacher_Attendance ASC LIMIT 5

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,Average_Teacher_Attendance
Austin Business and Entrepreneurship Academy High School,0.00%
Eric Solorio Academy High School,0.00%
Federico Garcia Lorca Elementary School,0.00%
Mariano Azuela Elementary School,0.00%
Marvin Camras Elementary School,0.00%


### Eliminar el signo '%' del conjunto de resultados anterior para la columna Asistencia promedio de estudiantes




In [21]:
%sql SELECT NAME_OF_SCHOOL, REPLACE(Average_Teacher_Attendance,'%','') FROM CHICAGO_PUBLIC_SCHOOLS_DATA ORDER BY Average_Teacher_Attendance ASC LIMIT 5

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,"REPLACE(Average_Teacher_Attendance,'%','')"
Austin Business and Entrepreneurship Academy High School,0.00
Eric Solorio Academy High School,0.00
Federico Garcia Lorca Elementary School,0.00
Mariano Azuela Elementary School,0.00
Marvin Camras Elementary School,0.00


### Escuelas que tienen una asistencia estudiantil promedio inferior al 70%



In [49]:
%sql SELECT NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE FROM CHICAGO_PUBLIC_SCHOOLS_DATA \
    WHERE CAST (REPLACE(AVERAGE_STUDENT_ATTENDANCE,'%','') AS DOUBLE) < 70 order by AVERAGE_STUDENT_ATTENDANCE 

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%
Manley Career Academy High School,66.80%
Chicago Vocational Career Academy High School,68.80%
Roberto Clemente Community Academy High School,69.60%


### La inscripción universitaria total (College Enrollment) para cada área comunitaria




In [12]:
%sql SELECT COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,SUM(COLLEGE_ENROLLMENT) FROM CHICAGO_PUBLIC_SCHOOLS_DATA GROUP BY COMMUNITY_AREA_NAME ORDER BY COMMUNITY_AREA_NUMBER

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,SUM(COLLEGE_ENROLLMENT)
ROGERS PARK,1,4068
WEST RIDGE,2,8197
UPTOWN,3,4388
LINCOLN SQUARE,4,4132
NORTH CENTER,5,7541
LAKE VIEW,6,7055
LINCOLN PARK,7,5615
NEAR NORTH SIDE,8,3362
EDISON PARK,9,910
NORWOOD PARK,10,6469


### Las 5 áreas comunitarias con la menor inscripción universitaria total (college enrollment) ordenadas en orden ascendente




In [14]:
%sql SELECT COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,SUM(COLLEGE_ENROLLMENT) FROM CHICAGO_PUBLIC_SCHOOLS_DATA \
GROUP BY COMMUNITY_AREA_NAME ORDER BY COLLEGE_ENROLLMENT ASC LIMIT 5

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,SUM(COLLEGE_ENROLLMENT)
CLEARING,64,2085
WEST PULLMAN,53,3240
OAKLAND,36,140
NORWOOD PARK,10,6469
NORTH LAWNDALE,29,5146


### Las 5 escuelas con la puntuación de seguridad (safe score) más baja.



In [19]:
%sql SELECT NAME_OF_SCHOOL, SAFETY_SCORE FROM CHICAGO_PUBLIC_SCHOOLS_DATA \
WHERE SAFETY_SCORE !='None' ORDER BY  SAFETY_SCORE ASC LIMIT 5

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,SAFETY_SCORE
Edmond Burke Elementary School,1.0
Luke O'Toole Elementary School,5.0
George W Tilton Elementary School,6.0
Foster Park Elementary School,11.0
Emil G Hirsch Metropolitan High School,13.0


### El índice de dificultades (hardship index) para el área comunitaria que tiene college enrollment de 4368




In [20]:
%sql SELECT hardship_index FROM CENSUS_DATA CD, CHICAGO_PUBLIC_SCHOOLS_DATA CPS WHERE CD.community_area_number = CPS.community_area_number AND college_enrollment = 4368

 * sqlite:///RealWorldData.db
Done.


HARDSHIP_INDEX
6.0


### El índice de dificultades (hardship index) para el área comunitaria que tiene el valor más alto para la inscripción universitaria (college enrollment)




In [23]:
%sql SELECT community_area_number, community_area_name,hardship_index FROM CENSUS_DATA CD, CHICAGO_PUBLIC_SCHOOLS_DATA CPS \
WHERE CD.community_area_number= CPS.community_area_number AND COLLEGE_ENROLLMENT =(SELECT MAX(COLLEGE_ENROLLMENT) FROM CHICAGO_PUBLIC_SCHOOLS_DATA)


 * sqlite:///RealWorldData.db
Done.


HARDSHIP_INDEX
6.0
